# <ins>Dash application</ins>: **Longitudinal modelling of the co-development of depression and cardio-metabolic risk from childhood to young adulthood**

In [1]:
import pyreadr
import pandas as pd
import numpy as np
import itertools

from dash import Dash, html, dcc, callback, Output, Input, State, ctx, dash_table
import dash_bootstrap_components as dbc
import dash_cytoscape as cyto

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# pd.set_option('display.max_rows', None)

In [2]:
PATH = '/Users/Serena/Desktop/panel_network/results/'

### Tab 1: generalized cross-lagged panel model 

Rscript 1 is designed to produce a single .RData file for each dep-cmr marker pair. This contains the following elements: 
- **`summ`**: a summary dataframe (with information about which marker is used and the timepoints included + mean ranges and number of observations)
- **`fit_meas`**: fit measures for every parameter combination, when the model converged.
- **`estimates`**: (unstandardized) estimates (+ bootsrapped SE, pvalues and CIs) for every parameter combination, when the model converged.
- **`failed`**: list of models that did not converge, with corresponding error or warning message.


In [3]:
def read_res1(depname, cmrname, path=PATH):
    res = pyreadr.read_r(f'{path}{depname}_{cmrname}.RData')
    summ = res['dat_summ']
    fitm = res['fit_meas'].T
    esti = res['estimates'].set_index('rep(f, nrow(es))')
    fail = list(res['failed'].index) # TODO: report warning message ...
    return(summ, fitm, esti, fail)
# use
# summ, fitm, esti, fail = read_res1('sDEP','FMI') 
# summ = read_res1('sDEP','BMI')[0]

First, `plot` the **median** and **interquartile ranges** of each measure included in the model, against time. This gives a more complete understanding of the data that is fed into the models.

In [4]:
def make_plot1(depname, cmrname):
    # load summary dataframe
    summ = read_res1(depname,cmrname)[0]
    
    # extract timepoints
    t_dep = [ float(x.split('_')[-1][:-1]) for x in summ.columns[:summ.shape[1]//2] ]
    t_cmr = [ float(x.split('_')[-1][:-1]) for x in summ.columns[summ.shape[1]//2:] ]

    # scatterplot function 
    def scat(t, name, fullname, shortname):
        means = summ.loc['Median', summ.columns.str.contains(name)]
        p = go.Scatter(x = t, y = means, 
                       error_y = dict(type='data', symmetric=False, # visible=True,
                                      array = summ.loc['3rd Qu.', summ.columns.str.contains(name)] - means,
                                      arrayminus = means - summ.loc['1st Qu.', summ.columns.str.contains(name)]),
                       name = fullname, text = [f'{shortname} {n}' for n in range(1,len(t)+1)],
                       marker = dict(size = 10, symbol = 'square',opacity = .8), opacity = .7,
                       hovertemplate = """ <b>%{text}</b> <br> Median: %{y:.2f} <br> Timepoint: %{x} years <br><extra></extra>""")
        return p

    fig = make_subplots(specs=[[{'secondary_y': True}]])
    
    fig.add_trace( scat(t_dep, depname, 'Depression score', 'DEP'), secondary_y=False)
    fig.add_trace( scat(t_cmr, cmrname, cmrname, cmrname), secondary_y=True )

    # Set y-axes
    def yrange(name):
        sub = summ.filter(like=name)
        ymin = sub.min(axis=1)['1st Qu.']; ymax = sub.max(axis=1)['3rd Qu.']
        range = ymax-ymin
        y_max_lower = ymax
        ymin = ymin - (range/10); ymax = ymax + (range/10)
        return [ymin, ymax, y_max_lower]
    
    fig.update_yaxes(title_text='<b>Depression score</b>', secondary_y=False, range=yrange(depname)[:2], 
                     mirror=True, ticks='outside', showline=True, linecolor='black', gridcolor='lightgrey')
    fig.update_yaxes(title_text=f'<b>{cmrname}</b>', secondary_y=True, range=yrange(cmrname)[:2], 
                     mirror=True, ticks='outside', showline=True, linecolor='black', gridcolor='lightgrey')
    # Set x-axis 
    fig.update_xaxes(title_text='Years', mirror=True, ticks='outside', showline=True, linecolor='black', gridcolor='lightgrey')

    # Group "cross-sectional" timepoints using grey background
    # ymin = summ.min(axis=1)['1st Qu.']-1; ymax = summ.max(axis=1)['3rd Qu.']+2
    crosspoints = []
    for i in range(len(t_dep)):
        xmin = min(t_dep[i], t_cmr[i])-.2; xmax = max(t_dep[i], t_cmr[i])+.2
        # rectangles 
        crosspoints.append( dict(x0 = str(xmin), x1 = str(xmax), y0 = 0, y1 = 1, xref='x', yref='paper', 
                                 type='rect', fillcolor='lightgray', opacity=.3, line_width=0, layer='below') )  
        # text 
        fig.add_trace( go.Scatter(x=[xmin + (xmax-xmin)/2], y=[yrange(depname)[2]], mode='text', text=i+1, 
                                  textposition='top center',
                                  textfont_size=13, textfont_color='dimgray', showlegend=False) )
    # Background
    fig.update_layout(# title = dict(text='Included measures\n', font=dict(size=15), automargin=True, yref='paper'),
                      plot_bgcolor='white', shapes=crosspoints, margin=dict(l=20, r=20, t=20, b=20))
    
    return(fig)

Find the best fitting model (i.e., lowest AIC) and describe its structure using the `model_structure` matrix.

In [5]:
# Define the matrix of all possible paramter combinations. 
model_structure = pd.read_csv(PATH+'../mats/model_structure.csv').set_index('Unnamed: 0')

# m = pd.DataFrame(itertools.product([0, 1], repeat=4)).T # all possible combinations
# m = m[m.sum().sort_values(ascending=False).index] # remove combinations not used and reorder
# m0 = m[m.columns[m.sum()!=3]].to_numpy(); m1 = np.ones(m0.shape, dtype=int) # 4x12 matrix
# mat1 = pd.DataFrame( np.vstack((m0,m1))); mat2 = pd.DataFrame( np.vstack((m1[:,1:],m0[:,1:])))

# # Construct the full matrix and rename rows and columns 
# model_structure = pd.concat([mat1,mat2],axis=1).rename(
#     columns={i:n for i,n in enumerate(['full_st','no_maCL_dep','no_maCL_cmr','no_maAR_dep','no_maAR_cmr',
#              'no_maCL','no_ma_dep','no_ma_CLcmr_ARdep','no_ma_CLdep_ARcmr','no_ma_cmr','no_maAR','no_ma',
#              'no_ltCL_dep','no_ltCL_cmr','no_ltAR_dep','no_ltAR_cmr','no_ltCL','no_lt_dep',
#              'no_lt_CLcmr_ARdep','no_lt_CLdep_ARcmr','no_lt_cmr','no_ltAR' ,'no_lt'])},
#     index={i:n for i,n in enumerate(['maCL_dep','maCL_cmr','maAR_dep','maAR_cmr','ltCL_dep','ltCL_cmr','ltAR_dep','ltAR_cmr'])})

# NOTE: this is the same as the mat matrix used to fit the models in Rscript 1.

def best_fit(depname, cmrname, list1=None):
    fitm = read_res1(depname, cmrname)[1]
    # Best fitting model (lowest AIC)
    mod = fitm.index[fitm.aic == fitm.aic.min()][0]
    if list1: # Return list of parameters estimated in the model 
        return list(model_structure.index[(model_structure[mod] > 0) & (model_structure.index.str.contains(list1))])
    else: # Return a dataframe with its name and model structure
        return pd.DataFrame(model_structure[mod])
    

Read in the results, select the best fitting model and construct its graph.
> `TODO`: make interactive parameter choice work and add fit measures to display 

In [6]:
def make_net1(depname, cmrname, which_model = 'best', width=1000):
    # read data
    summ,_,esti,fail = read_res1(depname, cmrname)
    
    # Best fitting model
    if which_model == 'best': modstr = best_fit(depname, cmrname).T
    elif which_model in fail: 
        return 'fail' # modstr = best_fit(depname, cmrname).T
    else: modstr = pd.DataFrame(model_structure[which_model]).T
    
    # Extract the estimated paramters from the result files
    def extr_est(name=None, which=None, lamb=False, eta_corr=False, imp_corr=False, model_output=esti):
        df = model_output.loc[modstr.index[0]]
        if lamb: l = list(round(df.loc[(df.lhs==f'eta_{name}')&(df.op=='=~'),'est'], 2))[which]
        elif eta_corr: l = df.loc[(df.lhs=='eta_dep') & (df.rhs=='eta_cmr'), 'est'][0]
        elif imp_corr: l = df.loc[ df.label.str.contains('comv'), 'est'][which]
        else:    l = list(round(df.loc[df.label.str.contains(name)].iloc[::-1]['est'], 2))[which]
        return(l)

    # Ready to draw
    nt = summ.shape[1]//2 # Number of timepoints
    
    pos_top = 30; pos_bot = 550 # Vertical cohordinates (in pixel)
    vs = ['dep','cmr']
    
    e = [] # Initialize
   
    # Eta factors nodes and correlations 
    for eta, pos in enumerate([pos_top, pos_bot]):
        e.append({'data': {'id':f'eta{eta}', 'label':'Eta'}, 'classes':'latent', 'position':{'x':width/2,'y':pos}})
        
    e.append({'data': {'source':'eta0', 'target':'eta1', 'firstname':'eta_corr', 'label':'%.2f' % extr_est(eta_corr=True) }})
    
    for i in range(1,nt+1): 
        
        e.append({'data': {'source':f'imp_dep{i}', 'target':f'imp_cmr{i}', 'firstname':'imp_corr', 
                           'label':'%.2f' % extr_est(which=i-1, imp_corr=True) }})
        
        for eta, v in enumerate(vs):
            
    # ===== Other nodes
            p = [pos_top+90, pos_top+190] if v=='dep' else [pos_bot-90,pos_bot-190] # define position
            e.extend([
                # Observed variables
                {'data': {'id':f'{v}{i}', 'label':f'{v.upper()} {i}'}, 'classes':'observed',
                 'position' : {'x':((width/nt)*i)-(width/nt)/2, 'y': p[0]} },
                # Impulses
                {'data': {'id':f'imp_{v}{i}', 'label':f'impulse {i}'}, 'classes':'latent',
                 'position' : {'x':((width/nt)*i)-(width/nt)/2, 'y': p[1]} }
            ])
        
    # ===== Edges: lambdas
            e.append({'data': {'source':f'eta{eta}', 'target':f'{v}{i}', 'firstname':'lambda',
                               'label':'%.2f' % extr_est(f'{v}',i-1, lamb=True) }})
            # impulses link
            e.append({'data': {'source':f'imp_{v}{i}', 'target':f'{v}{i}', 'firstname':'imp_link'}})
            
            if i < nt: 
                otherv = abs(eta-1)
                # maAR and AR terms
                if modstr[f'ltAR_{v}'][0]: e.append({'data': {'source':f'{v}{i}', 'target':f'{v}{i+1}',
                                   'weight': extr_est(f'^AR_{v}', i-1), 'label': f'AR{i}', 'firstname':'direct'}})
                if modstr[f'maAR_{v}'][0]: e.append({'data': {'source':f'imp_{v}{i}', 'target':f'{v}{i+1}',
                                   'weight': extr_est(f'^maAR_{v}',i-1), 'label': f'maAR{i}', 'firstname':'direct'}})
                # maCL and CL terms
                if modstr[f'ltCL_{v}'][0]: e.append({'data': {'source':f'{v}{i}', 'target':f'{vs[otherv]}{i+1}',
                                   'weight': extr_est(f'^CL_{v}', i-1), 'label': f'CL{i}', 'firstname':'direct'}})
                if modstr[f'maCL_{v}'][0]: e.append({'data': {'source':f'imp_{v}{i}', 'target':f'{vs[otherv]}{i+1}',
                                   'weight': extr_est(f'^maCL_{v}',i-1), 'label': f'maCL{i}', 'firstname':'direct'}})
                      
    return e

# ===================================================================================================================
# Also define the stile of the graph 
stylenet1=[ 
    # Nodes - shape & color
    {'selector':'.observed',
     'style':{'shape':'rectangle', 'height':25, 'width':60, 'border-width':2,'background-color':'white', 'border-color':'k', 
              'content':'data(label)','text-color':'grey','text-halign':'center','text-valign':'center'}},
    
    {'selector':'.latent', 
     'style':{'shape':'round', 'height':20, 'width':20, 'border-width':1,'background-color':'white', 'border-color':'silver'}},
    
    # Edges
    {'selector':'edge[firstname *= "direct"]', # directed paths 
     'style':{'curve-style':'straight', 'target-arrow-shape':'vee', 'width': 3, 'arrow-scale':1.2 }},
    
    {'selector':'edge[firstname *= "imp_link"]', # impulses links
     'style':{'curve-style':'straight', 'target-arrow-shape':'vee', 'width': 1, 'arrow-scale':.8 }},

    # Correlations
    {'selector':'edge[firstname *= "eta_corr"]',
     'style':{'curve-style':'unbundled-bezier','target-arrow-shape':'vee','source-arrow-shape':'vee', 'width': 1, 
              'control-point-distances': [-400,-500,-520,-500,-400],'control-point-weights': [0.01, 0.20, 0.5, 0.80, 0.99],
              'label':'data(label)','font-size':15, 'text-background-color':'silver', 'text-background-opacity':.7 }},
    
    {'selector':'edge[firstname *= "imp_corr"]', 
     'style':{'curve-style':'unbundled-bezier','target-arrow-shape':'vee','source-arrow-shape':'vee','width': 1, 
              'label':'data(label)','font-size':15, 'text-background-color':'silver', 'text-background-opacity':.7 }},
    
    # Lambdas
    {'selector':'edge[firstname *= "lambda"]', 
     'style':{'curve-style':'straight', 'target-arrow-shape':'vee', 'width': 1, 'arrow-scale':.8,
              'label':'data(label)','font-size':15, 'text-background-color':'silver', 'text-background-opacity':.7 }},
]
# Set the color of each edge type and the distance between sorce and label displaying its weight (to avoid overlapping) 
d = {'AR':['red',40],'maAR':['orange',70],'CL':['green',220],'maCL':['lightblue',40]}

for c in d.keys():
    stylenet1.append({'selector':f'[label *= "{c}"]', 
                      'style': {'line-color':d[c][0], 'target-arrow-color':d[c][0],
                                'source-label':'data(weight)', 'source-text-offset': d[c][1],'font-size':20, 'font-weight':'bold',
                                'text-background-color':d[c][0], 'text-background-opacity':.5 }
                     }) # 'source-label':'data(weight)','source-text-offset': 50,'source-endpoint':['-10%','0%']


In [7]:
# def make_table(depname, cmrname):
#     summ,_,_,_ = read_res1(depname,cmrname)
#     times = pd.DataFrame([['Depression']+[x.split('_')[-1] for x in summ.columns[:summ.shape[1]//2]], 
#                ['Cardio-metabolic risk']+[x.split('_')[-1] for x in summ.columns[summ.shape[1]//2:]]])
#     # .rename(columns = {0:'Marker'}, index={0:'dep',1:'cmr'})
#     return(times)

### Tab 2: cross-lagged panel network model
Rscript 2 fits and returns the .RData file for the longitudinal cross-lagged panel network model.

### Tab 3: cross-sectional network models
Rscript 3 is designed to produce a single .RData file for each single-timepoint, cross-sectional network model. This contains the following elements:
- **`wm`**: dataframe with all edge weights
- **`ci`**: 95% confidence intervals for those weights
- **`n_obs`**: number of observations the network is based on


In [8]:
def read_res3(time, path=PATH):
    res = pyreadr.read_r(f'{path}crosnet_{time}.RData')
    # weight matrix
    wm = res['wm']; wm['link'] = wm.index; wm[['a','b']] = wm.link.str.split(' ', expand = True)
    wm = wm.loc[wm.a!=wm.b, ] # remove links to between an edge and itself
    wm = wm.reset_index()[['a','b','V1']].rename(columns={'a':'node1','b':'node2','V1':'weight'})
    wm['dir'] = ['neg' if x<0 else 'pos' for x in wm.weight]
    # centrality indices
    ci = res['ci']; ci['class'] = ['dep' if t else 'cmr' for t in ci.node.str.contains('DEP')]
    # number of observations 
    nobs = int(res['n_obs'].iloc[0])
    return(wm, ci, nobs)

wm,ci,n = read_res3('9.6y-9.8y')

wm_trim = wm.loc[abs(wm.weight)>0.01,].reset_index(drop=True)

nodes = [{'data': {'id':node, 'label':node}, 'classes':group } 
       for node,group in ci[['node','class']].itertuples(index=False) ]
edges = [{'data': {'source':a, 'target':b, 'weight':w, 'width':round(abs(w)*20,2)}, 'classes':c} 
       for a,b,w,c in wm_trim.itertuples(index=False)]
net = nodes+edges

/var/folders/f9/fknlq25n23l74zmhg8zbckx80000gn/T/ipykernel_10523/3847059277.py:11: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  nobs = int(res['n_obs'].iloc[0])


## Set-up

#### App layout 
The application is structured into 3 main tabs. Add radio buttons to the app layout. 

#### Interactive graphs
I use the `plotly.express` library to build the interactive graphs. These are then assigned to the figure property of `dcc.Graph`, the compontent of the "Dash Core Components" module used to render interactive graphs.

#### Callback
Then, build the **callback** to create the interaction between the buttons and the chart. To work with the callback, import the callback module and the two arguments commonly used within the callback: Output and Input.

Both the RadioItems and the Graph components were given id names: used by the callback to identify the components.

The inputs and outputs of our app are the properties of a particular component. For example, input is the value property of the component that has the ID "controls-and-radio-item". Output is the figure property of the component with the ID "controls-and-graph", which is currently an empty dictionary (empty graph).

The callback function's argument col_chosen refers to the component property of the input. We build the chart inside the callback function. Every time the user selects a new radio item, the figure is rebuilt / updated. Return the graph at the end of the function. This assigns it to the figure property of the dcc.Graph, thus displaying the figure in the app.

In [9]:
def badge_it(text, color):
    return dbc.Badge(text, color=color, style={'padding':'4px 5px'})

In [10]:
app = Dash(__name__, external_stylesheets=[dbc.themes.LITERA], suppress_callback_exceptions=True)

app.layout = html.Div([
    # Title
    html.H1('Longitudinal modelling of the co-development of depression and cardio-metabolic risk from childhood to young adulthood',
             style={'textAlign':'center', 'font-weight':'bold'}),
    html.Br(), # space
    # Main body
    dbc.Row([
        dbc.Col(width=1), # add left margin
        dbc.Col([ 
            dcc.Tabs(id="tabs", value='tab-1', children=[
                dcc.Tab(label='Cross-lag panel model', value='tab-1'), # style={''}
                dcc.Tab(label='Cross-lag network analysis', value='tab-2'),
                dcc.Tab(label='Cross-sectional network analysis', value='tab-3') ]),
            html.Div(id='tabs-content') ]),
        dbc.Col(width=1), # add right margin
    ])
])

@callback(Output('tabs-content', 'children'), Input('tabs', 'value'))

def render_content(tab):
    if tab == 'tab-1':
        return  html.Div([ 
            html.Br(),
            html.Span(['This are the results of the generalized **cross-lag panel model** described as model 1 \
            in the paper. By default, the best fitting model is presented, but the paramter conbination can be \
            constumized.']),
            html.Hr(),
            # Input 
            dbc.Row([dbc.Col(width=1), # add left margin
                     dbc.Col([
                         html.H5(children='Depression score', style={'textAlign':'left'}),
                         dcc.RadioItems(id='dep-selection',
                                        options=[{'label': 'Self-reported', 'value': 'sDEP'},
                                                 {'label': 'Maternal report', 'value': 'mDEP'}],
                                        value='sDEP',
                                        inputStyle={'margin-left':'20px','margin-right':'20px'}),
                         html.Br(),
                         html.H5(children='Cardio-metabolic marker', style={'textAlign':'left'}),
                         dcc.Dropdown(id='cmr-selection', 
                                      options=[{'label': 'Fat mass index (FMI)', 'value': 'FMI'},
                                               {'label': 'Body mass index (BMI)', 'value': 'BMI'},
                                               {'label': 'Total fat mass', 'value': 'total_fatmass'},
                                               {'label': 'Waist circumference', 'value': 'waist_circ'}],
                                      value='FMI')
                     ]), 
                     dbc.Col(width=1), # add center space
                     dbc.Col([
                         html.H5(children='Model estimation', style={'textAlign':'left'}),
                         html.Div( style={'width':'50%','height':'65%','float':'left'},
                             children=[ dcc.Checklist(id ='lt-checklist', # className ='checkbox_1',
                                                      options=[{'label': html.Span([badge_it('AR', 'crimson'),' depression']), 
                                                                'value': 'ltAR_dep'},
                                                               {'label': html.Span([badge_it('AR', 'crimson'),' cardio-metab.']),
                                                                'value': 'ltAR_cmr'},
                                                               {'label': html.Span([badge_it('CL', 'green'),' depression']),
                                                                'value': 'ltCL_dep'},
                                                               {'label': html.Span([badge_it('CL', 'green'),' cardio-metab.']),
                                                                'value': 'ltCL_cmr'}],
                                                      inputStyle={'margin-left':'20px','margin-right':'20px'},
                                                      value=best_fit('sDEP', 'FMI', list1='lt'), 
                                                      labelStyle = {'display': 'block'}) ]),
                         html.Div( style={'width':'50%','height':'65%','float':'right'},
                             children=[ dcc.Checklist(id ='ma-checklist', # className ='checkbox_1',
                                                      options=[{'label': html.Span([badge_it('maAR', 'orange'),' depression']), 
                                                                'value': 'maAR_dep'},
                                                               {'label': html.Span([badge_it('maAR', 'orange'),' cardio-metab.']), 
                                                                'value': 'maAR_cmr'},
                                                               {'label': html.Span([badge_it('maCL', 'lightblue'),' depression']),
                                                                'value': 'maCL_dep'},
                                                               {'label': html.Span([badge_it('maCL', 'lightblue'),' cardio-metab.']),
                                                                'value': 'maCL_cmr'}],
                                                      inputStyle={'margin-left':'20px','margin-right':'20px'},
                                                      value=best_fit('sDEP', 'FMI', list1='ma'), 
                                                      labelStyle = {'display': 'block'}), 
                                       ]),
                         html.Div( style={'width':'25%','height':'35%','float':'right'},
                                children=[  dbc.Button('Update model', id='update-button', color='secondary', n_clicks=0,
                                    style={'font-weight':'bold', 'background-color':'silver','padding':'4px 10px'}) ] ),
                         html.Div(id='failed-model', style={'color':'red'}),
                     ]), 
                     # dbc.Col(width=1) # add right margin
                    ]),
            html.Hr(),
            # Table
            # dash_table.DataTable(id='time-table', data = make_table('sDEP', 'FMI').to_dict('records'), page_size=10),
            # Time plot 
            html.Div( dbc.Accordion([ dbc.AccordionItem([
                    html.Span("Variables included in the model:"),
                    dcc.Graph(id='time-graph', figure = make_plot1('sDEP','FMI'))],
                title="Inspect the variables included in this model")], start_collapsed=True)),
            # Network
            cyto.Cytoscape(id='cyto-graph',
                layout={'name': 'preset', 'fit':False},
                style={'width': '100%', 'height': '1000px'},
                minZoom=1, maxZoom=1, # reduce the range of user zooming 
                elements = make_net1('sDEP', 'FMI'), 
                stylesheet=stylenet1)
        ])
    
    elif tab == 'tab-3':
        return html.Div([
            html.Br(), 
            html.Span(['This are the results of the cross-sectional ', dbc.Badge('network model', color='crimson'),
                       ' described as a follow-up analysis in the paper. You can select the timpoint of interest below.']),
            html.Br(), 
            html.Hr(),
            html.Span('Select a timepoint:\n '),
            dcc.Slider(9.7, 25, step=None, value=9.7,
                       marks={ 9.7: {'label': '\n9.6-9.8 years', 
                                         'style': {'transform':'rotate(45deg)','whitespace':'nowrap'}},
                              10.5: {'label': '\n10.5 years', 
                                     'style': {'transform':'rotate(45deg)','whitespace':'nowrap'}},
                              15.5: {'label': ' 15.5 years'},
                              23.8: {'label': ' 23.8 years', 'style': {'color': '#f50'}} }, included=False ),
            
            cyto.Cytoscape(id='cros-net', 
                           layout={'name': 'cose', 'fit':True, 'padding':1, 
                                   'tilingPaddingVertical': 100,'tilingPaddingHorizontal': 100,
                                   'animation':False,
                                   'nodeRepulsion': 1000000, # 'gravity':0,'gravityRange':100,
                                   'nodeDimensionsIncludeLabels':True,
                                   # 'idealEdgeLength':0.0001,
                                   'minNodeSpacing':50},
                style={'width': '80%','height': '100%','position': 'absolute',
                       'left': 0,'top': 390,'z-index': 999},
                           # style={'width': '80%', 'height': '700px'},
                minZoom=1.1, maxZoom=1.1, # reduce the range of user zooming 
                elements = net, 
                stylesheet=[
                    {'selector': 'node', 'style': {'label': 'data(label)'} },
                     # Edge opacty and width
                    {'selector': 'edge', 'style': {'opacity': 'data(weight)', 'width': 'data(width)'}},
                    # Color nodes by group
                    {'selector': '.dep', 'style': {'background-color': 'lightblue'} },
                    {'selector': '.cmr', 'style': {'background-color': 'pink'} },
                    # Color edges by positive/negative weights
                    {'selector': '.neg', 'style': {'line-color': 'red'} },
                    {'selector': '.pos', 'style': {'line-color': 'blue'} },
                   
                ])
        ])

# Add controls to build the interaction: variable selection 
@callback(
    Output('time-graph', 'figure'),
    Input('dep-selection', 'value'),
    Input('cmr-selection', 'value')
)
def update_time_plot(dep_selection, cmr_selection):
    return make_plot1(dep_selection, cmr_selection)

@callback(
    Output('cyto-graph', 'elements'),
    # Output('failed-model','children'),
    Input('dep-selection', 'value'),
    Input('cmr-selection', 'value'),
    Input('update-button', 'n_clicks'),
    State('lt-checklist', 'value'),
    State('ma-checklist', 'value'),
    prevent_initial_call=True
)

def update_graph(dep_selection, cmr_selection, n_clicks, lt_checklist, ma_checklist):
    
    if ctx.triggered_id == 'update-button':
        
        checked = lt_checklist + ma_checklist
        series = pd.Series([ 1 if x in checked else 0 for x in model_structure.index], index=model_structure.index)
        model_name = model_structure.columns[ model_structure.eq(series, axis=0).all() ][0]
        
        update = make_net1(dep_selection, cmr_selection, which_model=model_name)
        
        # if update == 'fail': # prevents any single output updating
        #     return no_update, 'Model did not converge' 
        
        return update
    
    return make_net1(dep_selection, cmr_selection)
    

# @callback(
#     Output('time-table', 'data'),
#     Input('dep-selection', 'value'),
#     Input('cmr-selection', 'value')
# )
# def update_table(dep_selection, cmr_selection):
#     return make_table(dep_selection, cmr_selection).to_dict('records')


if __name__ == '__main__':
    app.run(debug=True, jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/
